# Import Modules

In [ ]:
import pandas as pd
import numpy as np

# Read in Data

In [ ]:
df = pd.read_csv("data/Student_Performance.csv")

# recoding categorical variable
df["Extracurricular Activities"] = df["Extracurricular Activities"].replace({"Yes": 1, "No": 0}).astype(int)

# randomly shuffle rows in df, for fair train-test split
df = df.sample(frac=1, random_state=123).reset_index(drop=True)

feature_matrix = df.to_numpy()

# Linear Regression

Aim: We want to create a  multiple linear regression model predict the `performance index` values for observations from unseen samples. 

 - This involves optimally fitting a straight-line to our features in some $d$-dimensional space 
    - 5 dimensions in our case, since we have 5 features in our data set and 1 target variable.

This multiple linear regression model takes the form:

$$ \hat{y}=Xw + b = w_{1}x_{1} + \dots + w_{d}x_{d} + b $$

Our estimates for `performance index` are $\hat{y} \in \mathbb{R}^{n}$ 

 - The notation $\hat{y} \in \mathbb{R}^{d}$ means that $\hat{y}$ is a $d$-dimensional vector, where each element of the vector is some real number 
     - $\mathbb{R}$ is just mathematical notation for "the set of all real numbers", and $\in$ means "$\hat{y}$ is a member of the set $\mathbb{R}^{d}$ ".

$$ \hat{y} = \begin{bmatrix} \hat{y_{1}} = x_{1}^{\top}w+b \\ \hat{y_{2}} = x_{2}^{\top}w+b \\ \vdots \\ \hat{y_{d}} = x_{d}^{\top}w+b \end{bmatrix} $$

 - As such, each element of the $\hat{y}$ vector corresponds to the estimate for each `performance index` value of each observation in our data, using a straight-line equation.
 - $x_{d}^{\top}$ refers to the transpose of each column from our design matrix. As such, these are row vectors which correspond to the set of values for each feature for a given observation in our data set.

Our design matrix is $X \in \mathbb{R}^{n \times d}$

$$ X = \begin{bmatrix} x_{11} & x_{12} & \dots & x_{1d} \\ x_{21} & x_{22} & \dots & x_{1d} \\ \vdots & \vdots & \ddots & \vdots \\ x_{n1} & x_{n2} & \dots & x_{nd} \\ \end{bmatrix} $$

Our optimal weights are $w \in \mathbb{R}^{d}$ 

$$ w = \begin{bmatrix} w_{1} \\ w_{2} \\ \vdots \\ w_{d} \end{bmatrix} $$

Our bias constant is given by $b \in \mathbb{R}$

## Simplfying the Formula: Vectorisation

Currently, the regression model takes the form:

$$
\hat{y} = Xw+b
$$

This is somewhat cumbersome to calculate as it will require optimising $w$ and $b$ separately. It would instead be better to optimise them simultaneously. We can achieve this via vectorisation. We can combine the bias constant into our weight vector and augment our design matrix by appending a column of $1\text{s}$. 

The bias constant is our $y$-intercept for our straight line which we are trying to fit to our data. 

Geometrically, this is equivalent to translating the straight line up or down according to the value of $b$. Generalised to more than 2 dimensions, this straight line is actually a hyperplane in $d$-dimensions, but the same logic in 2-dimensions still applies.

We can therefore think of this bias term as another weight in our model, but not one which rotates the hyperplane, but instead just translates it. Since this bias only translates the hyperplane, not rotates, it must move every point on the hyperplane uniformly in the same direction (whereas a rotation of the hyperplane would move different points by varying amounts - imagine rotating a line and looking at how far points near the axis of rotation move compared to those further away). 

Thus, to combine this bias term into our weights vector, we need to provide a constant dimension in our design matrix for the bias term to act along, hence the column of $1\text{s}$.

Doing this makes calculating $\hat{y}$ more straightforward as it is now a simple matrix-vector multiplication. 

$$ \hat{y} = \tilde{X}\tilde{w} $$

# Finding the Optimal Weights (+ Bias): Gradient Descent

To find the optimal values for the elements in $\tilde{w}$, we need a way to measure how close our estimates made using $\tilde{w}$ are from the correct values. We can use some pre-existing vector of labels $y$ to calibrate our model, but we need to be able to measure how far away our predictions are.

Linear regression typically uses the mean squared error (MSE) as a loss function

$$
\text{MSE} = \frac{1}{n}\sum_{i=1}^{n}(\tilde{X}\tilde{w}-y)^{2}
$$


The smaller the MSE, the closer our predictions $\hat{y}$ are to the correct values $y$ on average.

We can start by choosing a random set of weights and seeing how well we do (most likely terribly). We can then determine how our weights should be adjusted to minimise the MSE function through differentiating this loss function with respect to our weights.

$$
\frac{\partial L}{\partial w} = \frac{2}{n}\tilde{X}^{\top}(\tilde{X}\tilde{w}-y)
$$

We can then subtract some multiple $\eta$ (called the learning rate) of this derivative from the weights to get a new weight vector which will produce a smaller MSE. Iterating this process until the MSE stops reducing by any meaningful amount will allow us to converge on an estimate for the optimal weight vector.

$$\tilde{w}^{(t+1)} = \tilde{w}^{(t)} - \eta \frac{2}{n}\tilde{X}^{\top}(\tilde{X}\tilde{w}^{(t)}-y)$$

We how have all the pieces we need to begin creating a linear model.

# Building the Model

We can obtain $y$ and $X$ by subsetting our data frame.

In [ ]:
y = feature_matrix[:, -1:]
X = feature_matrix[:, :-1]

It is also important that our features are scaled. Without scaling, features which are expressed at higher order of magnitude (e.g. 1,000s rather than 10s) will be disproportionately weighted.

We can use min-max scaling as a method to normalise our data between 0 and 1:

$$
X^{'} = \frac{X-X_{\text{min}}}{X_{\text{max}}-X_{\text{min}}}
$$

In [ ]:
def min_max_scaling(X):
    min_vals = X.min(axis=0)
    max_vals = X.max(axis=0)
    ranges = max_vals - min_vals
    
    ranges[ranges == 0] = 1
    
    return (X - min_vals) / ranges

X = min_max_scaling(X)

We can also initialise our weight vector and bias term, which we will go on to optimise later. 

Recall that $w \in \mathbb{R}^{d}$. As such, our weight vector needs to have $d$ dimensions, where $d$ is the number of predictor variables in our data set (i.e. 5). We will also need our sample size $n$ in a moment, so we can just define these values now using our design matrix (since $X \in \mathbb{R}^{n \times d}$), alongside initialising $w$ and $b$.

In [ ]:
n = X.shape[0]
d = X.shape[1]

np.random.seed(123) # seed for reproducibility

w = np.random.rand(d, 1) # a column vector of d=5 real numbers drawn uniformly between 0 and 1.

b = np.random.rand(1, 1)

Recall that we want to obtain $\tilde{X}$ and $\tilde{w}$ for our vectorised calculations of $\hat{y}$. Thus, we need to append $b$ to $w$ and append a column of $1\text{s}$ to $X$

In [ ]:
tilde_w = np.vstack([w, b])

tilde_X = np.hstack([X, np.ones((n, 1))])

We can make some predictions $\hat{y}$ on a subset of our data and compare them to the correct values $y$. We can then optimise our weights (and bias) via gradient descent and then test the performance of our constructed model on another subset of our data which the model has not yet seen.

Thus, we can subset $y$ and $X$ into training and testing subsets. We want to use a fairly large proportion of our data for training, but leave a large enough proportion to test our model's performance. and $80$/$20$ split is typically used for linear regression, but this can be altered.

In [ ]:
train_size = 0.8
test_size = round(1 - train_size, 1) # round(x, 1) avoids float-point errors

X_train = tilde_X[:int(n * train_size)]
X_test = tilde_X[int(n * train_size):]

y_train = y[:int(n * train_size)]
y_test = y[int(n * train_size):]

With our train and test subsets created, we can begin training the model.

To find the MSE of our estimates, we can create a function to calculate the MSE.

In [ ]:
def _mean_squared_error(X, w, y):
    residuals = np.matmul(X, w) - y
    return np.mean(residuals ** 2)

Of course, the initial MSE is awful...

In [ ]:
_mean_squared_error(X=X_train, w=tilde_w, y=y_train)

np.float64(3224.742047147574)

We now need to perform gradient descent to optimise our weights and bias.

Thus, we need to create a function which is the derivative of the MSE, with respect to the weights (+ bias)

In [ ]:
def mse_derivative(X, w, y):
    residuals = np.matmul(X, w) - y
    return (2 * np.matmul(X.T, residuals)) / X.shape[0]

The gradient descent algorithm itself therefore takes 2 hyperparameters: the learning rate $\eta$ and the threshold below which we no longer consider decreases in the MSE to be meaningful.

In [ ]:
def gradient_descent(X, w, y, learning_rate, threshold, show_iterations):

    MSE_change = np.inf
    MSE_previous = np.inf
    MSE_history = []
    i = 0

    while MSE_change > threshold:

        MSE = _mean_squared_error(X=X, w=w, y=y)

        if i % 100 == 0:
            MSE_history.append((i, MSE))

        if i % 10_000 == 0 and show_iterations:
            print(f"{i:,} iterations \n MSE = {MSE}")

        MSE_derivative = mse_derivative(X=X, w=w, y=y)

        w = w - (learning_rate * MSE_derivative)

        MSE_change = abs(MSE_previous - MSE)
        MSE_previous = MSE

        i += 1

    return {
        "weights": w, 
        "iterations": i, 
        "mse": MSE,
        "mse_history": MSE_history
    }

In [ ]:
gd = gradient_descent(X=X_train, w=tilde_w, y=y_train, learning_rate=0.0001, threshold=1e-6, show_iterations=True)

0 iterations 
 MSE = 3224.742047147574
10,000 iterations 
 MSE = 180.445856508557
20,000 iterations 
 MSE = 124.33559389380241
30,000 iterations 
 MSE = 88.02369663867096
40,000 iterations 
 MSE = 63.62648877622798
50,000 iterations 
 MSE = 46.85894832003786
60,000 iterations 
 MSE = 35.14847853187601
70,000 iterations 
 MSE = 26.870266716184226
80,000 iterations 
 MSE = 20.95995165354552
90,000 iterations 
 MSE = 16.702663136203828
100,000 iterations 
 MSE = 13.610081413909462
110,000 iterations 
 MSE = 11.344697140964094
120,000 iterations 
 MSE = 9.671188523144123
130,000 iterations 
 MSE = 8.424307780377148
140,000 iterations 
 MSE = 7.487268051639443
150,000 iterations 
 MSE = 6.7770153916612195
160,000 iterations 
 MSE = 6.2341073695149944
170,000 iterations 
 MSE = 5.815714598378909
180,000 iterations 
 MSE = 5.490757831712012
190,000 iterations 
 MSE = 5.2365142266614235
200,000 iterations 
 MSE = 5.036238885291108
210,000 iterations 
 MSE = 4.877490641688946
220,000 iterations

In [ ]:
gd["mse"]

np.float64(4.259191635278776)

With our trained model, we can therefore make some predictions using our test data set and assess their performance using MSE. We should expect our MSE for our test set to be approximately equal to that of our training set.

In [ ]:
def predict(X, y, w):

    y_hat = np.matmul(X, w)

    MSE = _mean_squared_error(X=X, w=w, y=y)

    return {
        "y_hat": y_hat, 
        "mse": MSE
    }

In [ ]:
predictions, mse = predict(X_test, y_test, gd["weights"]).values()

print(
f"""
Predicted y values: \n{predictions}\n
Mean Squared Error: {mse:.4f}
"""
)


Predicted y values: 
[[53.97328862]
 [66.31328763]
 [39.2884766 ]
 ...
 [66.7781658 ]
 [26.74281689]
 [34.22512967]]

Mean Squared Error: 3.9474



In [ ]:
np.save("lm_model.npy", gd["weights"])

## Comparison with scikit-learn

Below, we can see that the Numpy-only method converges to an almost identical weight vector as with scikit-learn, albeit the run-time for scikit-learn is much faster and scikit-learn achieved a slightly better mean squared error. 

The main difference is that scikit-learn does not combine the bias terms into the weight vector as we did, hence the additional $0.0$ appended to the scikit-learn coefficients.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lm = LinearRegression().fit(X_train, y_train)
sklearn_lm_predictions = lm.predict(X_test)

print(
f"""
LR from Scratch: {[float(weight[0]) for weight in gd["weights"]]}
MSE: {mse}

Scikit-Learn: {[float(weight) for weight in lm.coef_[0]], float(lm.intercept_[0])}
MSE: {mean_squared_error(y_test, sklearn_lm_predictions)}
"""
)


LR from Scratch: [22.545121571676734, 59.58411303753044, 0.5292576989405147, 2.200442850320791, 1.475131183247628, 12.1431927656831]
MSE: 4.259191635278776

Scikit-Learn: ([22.849962262659247, 60.110817243338865, 0.6113781845223016, 2.4454624841957004, 1.754366681639631, 0.0], 11.388174475736953)
MSE: 3.929468330053579

